In [1]:
#clone repository

!git clone -b AI_2 https://github.com/SWhack-PhotoWithAI/PWA_APP.git
%cd PWA_APP/

!pip install flask-ngrok
!pip install -r NIMA/requirement.txt

Cloning into 'PWA_APP'...
remote: Enumerating objects: 537, done.
remote: Counting objects: 100% (537/537), done.
remote: Compressing objects: 100% (355/355), done.
remote: Total 1148 (delta 213), reused 403 (delta 123), pack-reused 611
Receiving objects: 100% (1148/1148), 63.11 MiB | 26.89 MiB/s, done.
Resolving deltas: 100% (314/314), done.
/content/PWA_APP
  Created wheel for random-name: filename=random_name-0.1.1-cp36-none-any.whl size=5474 sha256=b6922e546bf291e4e8c295af98eafaff7b60e41d2adedfcc8bd16a06b2a6a31d
  Stored in directory: /root/.cache/pip/wheels/17/fc/c5/abe2d2ee67eef0c0606d96ea02157c6c8079e32418da62b942
Successfully built random-name


In [ ]:
#run Flask WebServer

#Vision Module
from NIMA.evaluate_NIMA import evaluate
from Human.face_based import main
from Human.rt_eval import main as rt_main
#Flask 

from flask_ngrok import run_with_ngrok
# from flask import Flask
from flask import Flask, render_template, request, jsonify
from flask import send_file

from werkzeug.utils import secure_filename

import json
import os
import shutil
import requests
import glob

def get_image_list(image_dir):
    if image_dir is not None:
        print("Loading images from directory : ", image_dir)
        images = glob.glob(image_dir + '/*.png')
        images += glob.glob(image_dir + '/*.jpg')
        images += glob.glob(image_dir + '/*.jpeg')

    else:
        raise RuntimeError('Either -img_dir arguments must be passed as argument')

    return images

if 'images' not in os.listdir():
  os.mkdir('images')

else:
  imgDir_path = 'images/'
  img_list = get_image_list(imgDir_path)
  print(img_list)
  for img_path in img_list:
    os.remove(img_path)

app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run

@app.route('/predict_background', methods = ['POST'])
def predict_background(): #image 파일은 post형식으로 넘어오기 때문에
    #제일 score값이 좋은 image name을 return 하면 될듯.
    img_dir = 'images/'
    files = request.files.getlist('image') #이미지 여러개 받아오도록 해야함
    order_dict = {}
    index = 0
    for file in files:
        img_path = img_dir + secure_filename(file.filename)
        order_dict[img_path] = index
        index += 1 
        file.save(img_path)
    
    result = evaluate()
    result_index = order_dict[result['name']]

    _dict = {}
    _dict['index'] = result_index
    return _dict

@app.route('/predict_person', methods = ['POST'])
def predict_person():
    img_dir = 'images/'
    files = request.files.getlist('image') #이미지 여러개 받아오도록 해야함
    
    order_dict = {}
    index = 0
    for file in files:
        img_path = img_dir + secure_filename(file.filename)
        order_dict[img_path] = index
        index += 1
        file.save(img_dir + secure_filename(file.filename))

    result = main()
    best_img = ''
    max_score = -1
    for img_name, score in result.items():
        if score > max_score:
            best_img = img_name

    result_index = order_dict[best_img]

    _dict = {}
    _dict['index'] = result_index
    return _dict

@app.route('/predict_person_rt', methods = ['POST'])
def predict_person_rt():
    img_dir = 'images/'
    files = request.files.getlist('image') #이미지 여러개 받아오도록 해야함

    for file in files:
        img_path = img_dir + secure_filename(file.filename)
        file.save(img_dir + secure_filename(file.filename))

    result = rt_main()

    _dict = {}
    _dict['sen'] = result
    return _dict

@app.route('/cartoonization', methods = ['POST'])
def cartoonization():
    img_paths = []
    img_dir = 'images/'
    file = request.files.get('source') #이미지 여러개 받아오도록 해야함
    img_path = img_dir + secure_filename(file.filename)
    print(img_path)
    file.save(img_path)

    #return single image
    path = img_dir + 'test.jpg'
    url = 'https://master-white-box-cartoonization-psi1104.endpoint.ainize.ai/predict'
    _file = [
              ('source', (img_path, open(img_path, 'rb'), 'image/jpg'))
    ]
    _data = {'file_type' : 'image' }
    _response = requests.post(url ,data = _data, files=_file)
    print(_response)
    if _response.status_code == 200:
        with open(path, 'wb') as f:
            f.write(_response.content)
            f.close()

        return send_file(path, mimetype = 'image/jpeg')

    _dict = {}
    _dict['error'] = 'True'
    return _dict
    
@app.route("/")
def home():
    return "<h1>Running Flask on Google Colab!</h1>"
  
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://00deb3958743.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
Loading images from directory :  images/
img_path : images/person.jpg
removed image : images/person.jpg
img_score :  117.25374174118042
img_path : images/person2.jpg


127.0.0.1 - - [05/Feb/2021 20:13:51] "POST /predict_person HTTP/1.1" 200 -


removed image : images/person2.jpg
img_score :  177.6340063214302
Loading images from directory :  images/
img_path : images/person.jpg


127.0.0.1 - - [05/Feb/2021 20:14:11] "POST /predict_person HTTP/1.1" 200 -


removed image : images/person.jpg
img_score :  117.25374174118042


[AI_2 97f7f6b] modify post return oper
 1 file changed, 64 insertions(+), 13 deletions(-)
 rename runFlask.ipynb => run_server.ipynb (67%)
Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 2.08 KiB | 2.08 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/SWhack-PhotoWithAI/PWA_APP.git
   1cbcaad..97f7f6b  AI_2 -> AI_2
